In [1]:
import pandas as pd 
import geopy
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import certifi
import ssl 
from geopy.exc import GeocoderTimedOut
import time
import numpy as np
import json

In [5]:

# df = pd.read_csv('/Users/oussamanaji/Downloads/vo_vehicle (1).csv', encoding="ISO-8859-1", sep=";")
# df["ville"] = df["ville"].apply(lambda x: "0"*(5 - len(str(x))) + str(x))

# # Fonction pour obtenir latitude et longitude
# def get_lat_long(location_name):
#     ssl_context = ssl.create_default_context(cafile=certifi.where())
#     geolocator = Nominatim(user_agent="france_zip_code_locator", ssl_context=ssl_context)
#     try:
#         location = geolocator.geocode(f"{location_name}, France")
#         if location:
#             return location.latitude, location.longitude
#     except GeocoderTimedOut:
#         print("Geocoding service timed out.")
#     return None


# # Extraire les codes postaux uniques
# unique_postal_codes = df['ville'].unique()

# # Créer un dictionnaire pour le mapping des coordonnées
# postal_code_coordinates = {}
# for postal_code in unique_postal_codes:
#     coordinates = get_lat_long(postal_code)
#     if coordinates:
#         postal_code_coordinates[postal_code] = coordinates

# # Afficher le dictionnaire
# print(postal_code_coordinates)

{'14000': (49.18477625, -0.3739270766310739), '50400': (48.82908075, -1.5258293400000016), '62140': (50.3774973, 2.0093120631909382), '80330': (49.8587955, 2.3377605070352185), '62600': (50.40472805, 1.579823299328948), '14100': (49.13726275, 0.29670585002871885), '14600': (49.38317205, 0.2606336233272519), '14500': (48.829509849999994, -0.8659866045998419), '76600': (49.495932082978875, 0.1278343727088289), '80100': (50.1101639, 1.829265923951658), '62130': (50.3780011, 2.2975024643764006), '50300': (48.695174800000004, -1.3188984198413856), '02100': (49.8655251, 3.293217205110838), '76360': (49.5472075, 0.9216269829519484), '60880': (49.3788289, 2.7503926909621903), '60400': (49.55407945, 3.0225960059668333), '76000': (49.445400449999994, 1.0938964773934732), '27300': (49.10335845, 0.5847723507366971), '14230': (49.31284885, -1.071258364043361), '14390': (49.2655211, -0.1451122229392398), '76370': (49.93152925, 1.1652924140514465), '28400': (48.317747499999996, 0.8679515658498911), '

In [2]:
# Fonction pour obtenir latitude et longitude
def get_lat_long(location_name):
    ssl_context = ssl.create_default_context(cafile=certifi.where())
    geolocator = Nominatim(user_agent="france_zip_code_locator", ssl_context=ssl_context)
    try:
        location = geolocator.geocode(f"{location_name}, France")
        if location:
            return location.latitude, location.longitude
    except GeocoderTimedOut:
        print("Geocoding service timed out.")
    return None

with open('postal_code_coordinates.json', 'r') as file: #lire
    postal_code_coordinates = json.load(file)

def calculate_distance(row, client_location):
    vehicle_coords = postal_code_coordinates.get(row['ville'])
    if not vehicle_coords or len(vehicle_coords) != 2:
        # Gérer les cas où les coordonnées sont manquantes ou incorrectes
        return float('inf')
    
    return geodesic(client_location, vehicle_coords).kilometers

In [6]:



def recommandation(marque, modele, budget_max, zip_client, kilometrage_max):
    df = pd.read_csv('/Users/oussamanaji/Downloads/vo_vehicle (1).csv', encoding="ISO-8859-1", sep=";")
    df["ville"] = df["ville"].apply(lambda x: "0"*(5 - len(str(x))) + str(x))
    client_location = get_lat_long(zip_client)
    df['distance'] = df.apply(lambda row: calculate_distance(row, client_location), axis=1)
    #df = df.sort_values(by='distance')

    marque_modele_filtre = (df['marque'] == marque) & (df['modele'] == modele)

    df_filtered = df[marque_modele_filtre]

    budget_max = 15000  # Exemple de budget
    kilometrage_max = 100000  
    df_filtered = df_filtered[df_filtered['prix_ttc'] <= budget_max * 1.1]
    df_filtered = df_filtered[df_filtered['kilometres'] <= kilometrage_max * 1.05]

    return df_filtered[['ville', 'distance', 'marque', 'modele', 'prix_ttc']].head(10)
    



In [3]:
def recommandation_V2(marque, modele, budget_max, zip_client, kilometrage_max):
    df = pd.read_csv('/Users/oussamanaji/Downloads/vo_vehicle (1).csv', encoding="ISO-8859-1", sep=";")
    df["ville"] = df["ville"].apply(lambda x: "0"*(5 - len(str(x))) + str(x))
    client_location = get_lat_long(zip_client)
    df['distance'] = df.apply(lambda row: calculate_distance(row, client_location), axis=1)

    # Filtrage de la marque et du modèle
    marque_modele_filtre = (df['marque'] == marque) & (df['modele'] == modele)
    df_filtered = df[marque_modele_filtre]

    # Appliquer les contraintes de budget et de kilométrage
    df_filtered = df_filtered[df_filtered['prix_ttc'] <= budget_max * 1.1]
    df_filtered = df_filtered[df_filtered['kilometres'] <= kilometrage_max * 1.05]
    
    # Premier tour : Ville la plus proche
    result_1 = df_filtered.loc[df_filtered['distance'].idxmin()]
    print("Premier tour : Ville la plus proche")
    print(result_1[['ville', 'distance', 'marque', 'modele', 'prix_ttc']])

    # Deuxième tour : Trois villes les plus proches
    result_2 = df_filtered.nsmallest(3, 'distance')
    print("\nDeuxième tour : Trois villes les plus proches")
    print(result_2[['ville', 'distance', 'marque', 'modele', 'prix_ttc']])

    # Troisième tour : Villes dans un rayon de 50 km
    result_3 = df_filtered[df_filtered['distance'] <= 50]
    print("\nTroisième tour : Villes dans un rayon de 50 km")
    print(result_3[['ville', 'distance', 'marque', 'modele', 'prix_ttc']])

    # Quatrième tour : Toutes les villes
    result_4 = df_filtered
    print("\nQuatrième tour : Toutes les villes")
    print(result_4[['ville', 'distance', 'marque', 'modele', 'prix_ttc']])

In [4]:
budget_max = 15000  # Exemple de budget
kilometrage_max = 100000  
zip_client = '14800'
marque = 'PEUGEOT'
modele = '208'

print(recommandation_V2(marque, modele, budget_max, zip_client, kilometrage_max))

Premier tour : Ville la plus proche
ville         14800
distance        0.0
marque      PEUGEOT
modele          208
prix_ttc      13990
Name: 2367, dtype: object

Deuxième tour : Trois villes les plus proches
      ville   distance   marque modele  prix_ttc
2367  14800   0.000000  PEUGEOT    208     13990
3114  14800   0.000000  PEUGEOT    208     14990
1043  14600  12.125344  PEUGEOT    208     15490

Troisième tour : Villes dans un rayon de 50 km
      ville   distance   marque modele  prix_ttc
37    14100  25.530582  PEUGEOT    208     15990
43    14000  39.055646  PEUGEOT    208     13990
62    14000  39.055646  PEUGEOT    208     16490
65    14000  39.055646  PEUGEOT    208     16490
209   14100  25.530582  PEUGEOT    208      9990
219   14000  39.055646  PEUGEOT    208     16490
323   14000  39.055646  PEUGEOT    208     16490
462   14000  39.055646  PEUGEOT    208     15490
736   14000  39.055646  PEUGEOT    208     16490
747   14000  39.055646  PEUGEOT    208     16490
856   14

In [5]:
import folium
from IPython.display import display

all_coordinates = list(postal_code_coordinates.values())
avg_latitude = sum(coord[0] for coord in all_coordinates) / len(all_coordinates)
avg_longitude = sum(coord[1] for coord in all_coordinates) / len(all_coordinates)
map_center = [avg_latitude, avg_longitude]

# Créer une carte centrée sur les coordonnées calculées
my_map = folium.Map(location=map_center, zoom_start=6)

# Ajouter des marqueurs pour chaque code postal
for postal_code, coord in postal_code_coordinates.items():
    folium.Marker(location=coord, popup=f"Code postal: {postal_code}").add_to(my_map)

# Afficher la carte dans le notebook
display(my_map)